<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

#### What will you create in this notebook?

In this notebook, we will be creating a real time, interactive view of unit level data, combined with market level metrics. This notebook will work with zipcodes nationwide. 

<p align="center">
  <img src="../../../images/unit_level_dashboard.png" alt="Alt text">
</p>

#### Need help getting started?

You will need an upgraded Parcl Labs API account to use this notebook. You can register and upgrade your account [here](https://dashboard.parcllabs.com/signup).

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/housing_market_research/unit_level/unit_level_data.ipynb)

### 1. Import the Parcl Labs Python Library

In [1]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from parcllabs import ParclLabsClient
from parcllabs.beta.charting.utils import create_labs_logo_dict


client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=12 # set default limit
)

### 2. Retrieve Data

In [2]:
# set buy box criteria and zip code
ZIP_CODE = 30088
SQ_FT_MIN = 1000
SQ_FT_MAX = 2500
YEAR_BUILT_MIN = 1990
YEAR_BUILT_MAX = 2024
PROPERTY_TYPE = 'single_family'


unit_ids = client.property.search.retrieve(
    zip=ZIP_CODE,
    sq_ft_min=SQ_FT_MIN,
    sq_ft_max=SQ_FT_MAX,
    year_built_min=YEAR_BUILT_MIN,
    year_built_max=YEAR_BUILT_MAX,
    property_type=PROPERTY_TYPE
)

In [3]:
hist = client.property.events.retrieve(parcl_property_ids=unit_ids['parcl_property_id'].tolist())
hist['event_date'] = pd.to_datetime(hist['event_date'])

on 298: /opt/anaconda3/envs/parcllabs/lib/python3.11/site-packages/parcllabs/services/data_utils.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
          output = pd.concat(non_empty_dfs).reset_index(drop=True)


|████████████████████████████████████████| 298/298 [100%] in 0.5s (597.66/s) 


In [4]:
# retrieve market metrics
# now lets enrich this same map with trending rental rates. First lets get the market id
market_id = client.search.markets.retrieve(
    query=ZIP_CODE,
    location_type='ZIP5',
    limit=1
)

start_date = '2023-01-01'
yields = client.rental_market_metrics.gross_yield.retrieve(
    parcl_ids=market_id['parcl_id'].tolist(),
    property_type='SINGLE_FAMILY',
    limit=1
)

prices = client.market_metrics.housing_event_prices.retrieve(
    parcl_ids=market_id['parcl_id'].tolist(),
    # start_date=start_date,
    property_type='SINGLE_FAMILY',
    limit=1
)

rental_inventory = client.rental_market_metrics.new_listings_for_rent_rolling_counts.retrieve(
    parcl_ids=market_id['parcl_id'].tolist(),
    property_type='SINGLE_FAMILY',
    limit=1
)

rental_inventory_7_day = rental_inventory['rolling_7_day'].values[0]
rental_inventory_30_day = rental_inventory['rolling_30_day'].values[0]
gross_yields = yields['pct_gross_yield'].values[0]
rental_price = prices['price_median_new_rental_listings'].values[0]

|████████████████████████████████████████| 1/1 [100%] in 0.1s (9.36/s) 
|████████████████████████████████████████| 1/1 [100%] in 0.1s (10.28/s) 
|████████████████████████████████████████| 1/1 [100%] in 0.1s (9.97/s) 


### 3. Format Data

In [5]:
df = hist.copy()
df['rental'] = df['event_type'].apply(lambda x: 'Rentals' if 'RENTAL' in x else 'Sales or Listings')
total_number_of_bbox_rental_units = df[df['rental'] == 'Rentals'].shape[0]
total_number_of_bbox_recent_rentals = df[(df['rental'] == 'Rentals') & (df['event_date'] >= '6/1/2024')].shape[0]
vacancy_rate = total_number_of_bbox_recent_rentals / total_number_of_bbox_rental_units

In [6]:
# capture the last event date for each property
maxv = df.groupby('parcl_property_id')['event_date'].max().reset_index(name='event_date')
df = df.merge(maxv, on=['parcl_property_id', 'event_date'], how='inner')

In [7]:
# date format for text
df['event_date'] = df['event_date'].dt.strftime('%Y-%m-%d')

In [8]:
def hover_data(row):
    price = f"{int(row['price'])}" if not pd.isnull(row['price']) else ''
    return f"Date: {str(row['event_date']).split(' ')[0]}<br>Event Type: {row['event_type']}<br>Price: ${price}<br>Beds: {row['bedrooms']}<br>Baths: {row['bathrooms']}<br>Square Feet: {row['square_footage']}<br>Year Built: {row['year_built']}"

# tooltip data
df['hover_data'] = df.apply(hover_data, axis=1)

### 4. Plot Data

In [9]:
# Create the scatter mapbox figure
fig = px.scatter_mapbox(
    df,
    lat='latitude',
    lon='longitude',
    color='rental',
    color_continuous_scale=px.colors.cyclical.IceFire,
    hover_name='hover_data',
    size_max=10,
    zoom=13,
    mapbox_style='open-street-map'
)

# Update hover template
fig.update_traces(hovertemplate='%{hovertext}')

# Update layout
fig.update_layout(
    title='',
    legend=dict(
        title='',
        x=0.01,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.5)' 
    ),
    width=1000,  # Increased width to accommodate the text pane
    height=800,
    margin=dict(l=250, r=10, t=30, b=10),
)

# Add text annotations
x_annotation = -0.05

location_anchor = 1
bbox_anchor = 0.6
market_anchor = 0.85
annotations = [
    dict(
        text="Location:",
        x=x_annotation,
        y=location_anchor,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210,
        font=dict(size=16, color='blue')
    ),
    dict(
        text="Metro: Atlanta",
        x=x_annotation,
        y=location_anchor-0.05,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210,
        font=dict(size=14)
    ),
    dict(
        text=f"Zip: {ZIP_CODE}",
        x=x_annotation,
        y=location_anchor-0.1,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210,
        font=dict(size=14)
    ),
    dict(
        text="Buy Box Criteria:",
        x=x_annotation,
        y=bbox_anchor,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210,
        font=dict(size=16, color='blue')
    ),
    dict(
        text=f"Property Type: Single Family",
        x=x_annotation,
        y=bbox_anchor - 0.05,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210,
        font=dict(size=14)
    ),
    dict(
        text=f"Square Footage: {SQ_FT_MIN}-{SQ_FT_MAX} sq ft.",
        x=x_annotation,
        y=bbox_anchor - 0.1,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210, 
        font=dict(size=14)
    ),
    dict(
        text=f"Year Built: {YEAR_BUILT_MIN}-{YEAR_BUILT_MAX}",
        x=x_annotation,
        y=bbox_anchor - 0.15,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210,
        font=dict(size=14)
    ),
    dict(
        text=f"Vacancy Rate: {vacancy_rate:.2%}",
        x=x_annotation,
        y=bbox_anchor - 0.2,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210,
        font=dict(size=14)
    ),
    dict(
        text="Single Family Market Metrics:",
        x=x_annotation, 
        y=market_anchor,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210,
        font=dict(size=16, color='blue')
    ),
    dict(
        text=f"New Rentals (7 day): {rental_inventory_7_day}",
        x=x_annotation,
        y=market_anchor-0.05,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210,
        font=dict(size=14)
    ),
    dict(
        text=f"New Rentals (30 day): {rental_inventory_30_day}",
        x=x_annotation,
        y=market_anchor-0.1,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210,
        font=dict(size=14)
    ),
    dict(
        text=f"Current Rental Price: {rental_price}",
        x=x_annotation,
        y=market_anchor-0.15,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210,
        font=dict(size=14)
    ),
    dict(
        text=f"Gross Yield: {gross_yields}",
        x=x_annotation,
        y=market_anchor-0.2,
        xref='paper',
        yref='paper',
        showarrow=False,
        align='left',
        xshift=-210, 
        font=dict(size=14)
    ),
]

fig.update_layout(annotations=annotations)

# Add logo (assuming create_labs_logo_dict is defined elsewhere)
fig.add_layout_image(
    create_labs_logo_dict(
        color='blue',
        xanchor='left',
        yanchor='bottom',
        x=-0.29,
        y=0,
        sizex=0.25,
        sizey=0.25,
    ),
)

# Show figure
fig.show()
